In [2]:
import streamlit as st
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from PIL import Image

# Constants
IMG_SIZE = 224

# Load model
@st.cache_resource
def load_cataract_model():
    model = load_model("Final_cataract_model.h5")
    return model

model = load_cataract_model()

# Prediction function
def predict(image):
    img = image.resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    label = "Cataract" if prediction > 0.5 else "Normal"
    confidence = prediction if prediction > 0.5 else 1 - prediction
    return label, confidence

# Streamlit UI
st.title("🧠 Cataract Detection App")
st.write("Upload an eye image to detect if Cataract is present.")
st.caption("Developed by: Ravindra Singh")

# File upload
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    st.write("Predicting...")
    label, confidence = predict(image)

    st.success(f"**Prediction:** {label}")
    st.info(f"**Confidence:** {confidence:.2%}")


2025-09-01 11:57:30.812 
  command:

    streamlit run C:\Users\Ravindra Singh\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


# Live Testing Using Web Came

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Constants
IMG_SIZE = 224
THRESHOLD = 0.5

# Load model
model = load_model("Final_cataract_model.h5")

# Load Haar Cascades
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")

# Predict cataract from eye patch
def predict_eye(eye_img):
    eye_img = cv2.resize(eye_img, (IMG_SIZE, IMG_SIZE))
    eye_img = eye_img / 255.0
    eye_img = np.expand_dims(eye_img, axis=0)
    pred = model.predict(eye_img)[0][0]
    label = "Cataract" if pred > THRESHOLD else "Normal"
    confidence = pred if pred > THRESHOLD else 1 - pred
    return label, confidence

# Start webcam
cap = cv2.VideoCapture(0)
print("[INFO] Live Cataract Detection - Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    for (fx, fy, fw, fh) in faces:
        face_color = frame[fy:fy+fh, fx:fx+fw]
        face_gray = gray[fy:fy+fh, fx:fx+fw]

        # Detect eyes in face ROI
        eyes = eye_cascade.detectMultiScale(face_gray, scaleFactor=1.1, minNeighbors=6)

        # Sort eyes by x-position (left-to-right)
        eyes = sorted(eyes, key=lambda e: e[0])[:2]  # Take only 2 eyes max

        for (ex, ey, ew, eh) in eyes:
            # Filter: reject too small or very low/high detections
            if ew < fw * 0.15 or eh < fh * 0.15:
                continue
            if ey > fh * 0.6:  # Too low to be an eye
                continue

            eye_img = face_color[ey:ey+eh, ex:ex+ew]
            label, conf = predict_eye(eye_img)

            color = (0, 255, 0) if label == "Normal" else (0, 0, 255)
            cv2.rectangle(face_color, (ex, ey), (ex+ew, ey+eh), color, 2)
            cv2.putText(face_color, f"{label} ({conf*100:.2f}%)", (ex, ey-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Live Cataract Detection (Filtered Eyes)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Live Cataract Detection - Press 'q' to quit
1/1 [==============================] - 0s 47ms/step
